In [4]:
import pandas as pd
import os
import requests
import lxml.html

In [3]:
url = 'https://finance.naver.com/fund/fundFinderList.nhn?sortOrder=y1&search=AIN,T20,T21&pageSize=20&page={page}'

In [35]:
data = []

page = 1
while True:
    pg_url = url.format(page=page)
    html = requests.get(pg_url)
    doc = lxml.html.fromstring(html.content)
    trs = doc.cssselect("table.tbl_fund > tbody > tr")
    if len(trs) == 0: 
        break
    
    for tr in trs:
        tr_fund_name = tr.cssselect("table.tbl_fund_info thead tr th")
        fund_name = tr_fund_name[0].attrib['title']

        values = [fund_name]
        for td in tr[2:5]:
            if len(td.cssselect("strong")) == 0:
                values.append(td.text)
            else:
                values.append(td.cssselect("strong")[0].text)
        data.append(values)
    
    page += 1
    
df_result = pd.DataFrame(data, columns=['펀드명', '1개월수익률', '3개월수익률', '1년수익률'])
df_result.to_csv("fund_list.csv")